In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
device

device(type='cuda')

In [3]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Define the transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the full dataset from the root directory containing subfolders for each class
dataset = datasets.ImageFolder('/kaggle/input/assignment-4-dataset/images', transform=transform)

# Split the dataset into 80/20 for train/test
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size], generator=torch.Generator().manual_seed(42))

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [4]:
class MLPModel(nn.Module):
    def __init__(self):
        super(MLPModel, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(224 * 224 * 3, 4096)
        self.fc2 = nn.Linear(4096, 2048)
        self.fc3 = nn.Linear(2048, 1024)
        self.fc4 = nn.Linear(1024, 512)
        self.fc5 = nn.Linear(512, 2)  # Binary classification output
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.fc5(x)
        return x

mlp_model = MLPModel().to(device)


In [5]:
from torchvision.models import VGG16_Weights, VGG19_Weights

def build_vgg_model(version='VGG16', tune_all=True):
    if version == 'VGG16':
        weights = VGG16_Weights.DEFAULT
    else:
        weights = VGG19_Weights.DEFAULT
    
    base_model = models.vgg16(weights=weights) if version == 'VGG16' else models.vgg19(weights=weights)

    # Freeze convolution layers if not tuning all layers
    if not tune_all:
        for param in base_model.features.parameters():
            param.requires_grad = False

    # Modify the classifier for binary classification
    base_model.classifier[6] = nn.Linear(base_model.classifier[6].in_features, 2)  # Binary classification output
    return base_model.to(device)

# Build VGG models with different tuning strategies
vgg_tune_all = build_vgg_model(tune_all=True)
vgg_tune_mlp = build_vgg_model(tune_all=False)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 204MB/s]  


In [6]:
# Step 4: Training Function

def train_model(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(dataloader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

In [7]:
# Step 5: Training the Models

# Set up loss and optimizers
criterion = nn.CrossEntropyLoss()
optimizer_mlp = optim.Adam(mlp_model.parameters(), lr=0.0001)
optimizer_vgg_all = optim.Adam(vgg_tune_all.parameters(), lr=0.0001)
optimizer_vgg_mlp = optim.Adam(vgg_tune_mlp.parameters(), lr=0.0001)

# Train each model
print("Training MLP Model...")
train_model(mlp_model, train_loader, criterion, optimizer_mlp)

print("Training VGG (Full Tuning) Model...")
train_model(vgg_tune_all, train_loader, criterion, optimizer_vgg_all)

print("Training VGG (MLP Tuning) Model...")
train_model(vgg_tune_mlp, train_loader, criterion, optimizer_vgg_mlp)


Training MLP Model...
Epoch 1/10, Loss: 0.8542
Epoch 2/10, Loss: 0.7091
Epoch 3/10, Loss: 0.6548
Epoch 4/10, Loss: 0.6242
Epoch 5/10, Loss: 0.6281
Epoch 6/10, Loss: 0.8784
Epoch 7/10, Loss: 0.7276
Epoch 8/10, Loss: 0.5470
Epoch 9/10, Loss: 0.8032
Epoch 10/10, Loss: 0.7040
Training VGG (Full Tuning) Model...
Epoch 1/10, Loss: 0.2621
Epoch 2/10, Loss: 0.0082
Epoch 3/10, Loss: 0.0010
Epoch 4/10, Loss: 0.0985
Epoch 5/10, Loss: 0.1769
Epoch 6/10, Loss: 0.4319
Epoch 7/10, Loss: 0.0499
Epoch 8/10, Loss: 0.0712
Epoch 9/10, Loss: 0.0127
Epoch 10/10, Loss: 0.0047
Training VGG (MLP Tuning) Model...
Epoch 1/10, Loss: 0.3676
Epoch 2/10, Loss: 0.0093
Epoch 3/10, Loss: 0.0005
Epoch 4/10, Loss: 0.0001
Epoch 5/10, Loss: 0.0000
Epoch 6/10, Loss: 0.0000
Epoch 7/10, Loss: 0.0000
Epoch 8/10, Loss: 0.0000
Epoch 9/10, Loss: 0.0000
Epoch 10/10, Loss: 0.0000


In [8]:
# Step 6: Evaluation Function

def evaluate_model(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print(classification_report(all_labels, all_preds))
    print(confusion_matrix(all_labels, all_preds))

# Step 7: Evaluating the Models

print("MLP Model Evaluation:")
evaluate_model(mlp_model, test_loader)

print("VGG (Full Tuning) Model Evaluation:")
evaluate_model(vgg_tune_all, test_loader)

print("VGG (MLP Tuning) Model Evaluation:")
evaluate_model(vgg_tune_mlp, test_loader)

MLP Model Evaluation:
              precision    recall  f1-score   support

           0       0.78      0.28      0.41        25
           1       0.44      0.88      0.58        16

    accuracy                           0.51        41
   macro avg       0.61      0.58      0.50        41
weighted avg       0.64      0.51      0.48        41

[[ 7 18]
 [ 2 14]]
VGG (Full Tuning) Model Evaluation:
              precision    recall  f1-score   support

           0       0.92      0.96      0.94        25
           1       0.93      0.88      0.90        16

    accuracy                           0.93        41
   macro avg       0.93      0.92      0.92        41
weighted avg       0.93      0.93      0.93        41

[[24  1]
 [ 2 14]]
VGG (MLP Tuning) Model Evaluation:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        25
           1       1.00      1.00      1.00        16

    accuracy                           1.00        